In [1]:
pip install fsspec

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gcsfs

Note: you may need to restart the kernel to use updated packages.


In [7]:
import math

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow import keras
from tqdm.auto import tqdm

def smape_loss(y_true, y_pred):
    return 100 * (tf.abs(y_true - y_pred) / ((y_true + tf.abs(y_pred)) / 2))
class ModelBasic:
    def __init__(self,df_input):
        self.df_input = df_input

    def modelling(self):
        print(f"modelling...")
        raw_features = self.df_input.loc[
            :,
            [
                "ticker",
                "date",
                "rd_activo_news",
                # Las comento de momento ya que tendremos que replantear el modelo
                # y por otro lado, tendremos que evitar lo máximo posible el sesgo
                # de look-ahead
                # "r_d_menos_1",
                # "r_d_mas_1",
                # "beta",
                # "r_d_bench",
                # "r_adj_menos_1",
                # "r_adj_mas_1",
            ],
        ]
        raw_features["date"] = pd.to_datetime(raw_features.date, format="%Y-%m-%d")
        tickers = raw_features.ticker.unique()

        # One-hot encoding
        for ticker in tickers[1:]:
            raw_features[ticker] = raw_features.ticker == ticker

        # Seasonal variations (Fourier series)
        dayofyear = raw_features.date.dt.dayofyear
        for k in range(1, len(tickers)):
            raw_features[f"sin{k}"] = np.sin(dayofyear / 365 * 2 * math.pi * k)
            raw_features[f"cos{k}"] = np.cos(dayofyear / 365 * 2 * math.pi * k)
            raw_features[f"{tickers[k]}_sin{k}"] = (
                raw_features[f"sin{k}"] * raw_features[tickers[k]]
            )
            raw_features[f"{tickers[k]}_cos{k}"] = (
                raw_features[f"cos{k}"] * raw_features[tickers[k]]
            )

        raw_features = raw_features.drop(columns="ticker")
        raw_features = raw_features.drop(columns="date")
        # raw_features = pd.get_dummies(raw_features)

        features = raw_features.values
        # target = df_input.alpha_expost_label
        target = df_input.alpha_expost

        print(features, target)

        x_train, x_test, y_train, y_test = train_test_split(
            features, target, test_size=0.33, 
        )
        x_train = np.asarray(x_train).astype("float32")
        x_test = np.asarray(x_test).astype("float32")
        y_train = np.asarray(y_train).astype("float32")
        y_test = np.asarray(y_test).astype("float32")

        # revisar los inf
        x_train[np.isinf(x_train)] = 0
        x_test[np.isinf(x_test)] = 0
        y_train[np.isinf(y_train)] = 0
        y_test[np.isinf(y_test)] = 0

        scaler = StandardScaler()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

        # y_train = y_train.reshape(-1, 1)
        # y_test = y_test.reshape(-1, 1)

        # model = keras.Sequential()
        # model.add(keras.layers.Input(shape=(x_train.shape[1],), name="entrada"))
        # model.add(keras.layers.Dense(1, name="salida"))
        # model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01),
        #           loss='mean_squared_error',
        #           metrics=['acc'])

        model = keras.Sequential()
        model.add(keras.layers.Input(shape=(x_train.shape[1],), name="entrada"))
        model.add(
            keras.layers.Dense(units=64, kernel_regularizer=keras.regularizers.l2(0.01))
        )
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Activation("relu"))
        model.add(tf.keras.layers.Dropout(0.2))
        model.add(
            keras.layers.Dense(units=32, kernel_regularizer=keras.regularizers.l2(0.01))
        )
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Activation("relu"))
        model.add(tf.keras.layers.Dropout(0.2))
        model.add(keras.layers.Dense(1, name="salida"))

        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.01),
            # loss=smape_loss,
            loss="mean_squared_error",
            metrics=["acc"],
        )

        history = model.fit(x_train, y_train, epochs=200, batch_size=x_train.shape[0], validation_split=0.2)

        y_pred = model.predict(x_test)
        # y_pred = np.exp(model.predict(x_test))

        for i in range(10):
            print(f'y_test[{i}] : {y_test[i]} , y_pred[{i}] : {y_pred[i]}')

        smape = np.mean(smape_loss(y_test, y_pred))        
        print(f"SMAPE: {smape:.5f}")

In [8]:
df_input = pd.read_csv('gs://tfm_aideas_datasets/dataset.csv', sep=";")
print(df_input)
model = ModelBasic(df_input)
model.modelling()

       Unnamed: 0 ticker                           sector  \
0           36284    TEF  Tecnología y Telecomunicaciones   
1           23026   AENA             Servicios de Consumo   
2           32912   CABK            Servicios Financieros   
3            4142   BBVA            Servicios Financieros   
4           20395   BBVA            Servicios Financieros   
...           ...    ...                              ...   
30444       32852    MRL          Servicios Inmobiliarios   
30445       20327   BBVA            Servicios Financieros   
30446       30344   CABK            Servicios Financieros   
30447       15086    SAN            Servicios Financieros   
30448       37647   CABK            Servicios Financieros   

                        subsector                 date  \
0      Telecomunicaciones y Otros  2021-12-21 14:11:20   
1       Transporte y Distribución  2021-09-15 10:16:00   
2        Bancos y Cajas de Ahorro  2021-05-19 10:07:21   
3        Bancos y Cajas de Ahorro  

C:\Users\Tienda\AppData\Local\Temp/ipykernel_64644/1060488693.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  raw_features[f"{tickers[k]}_sin{k}"] = (
C:\Users\Tienda\AppData\Local\Temp/ipykernel_64644/1060488693.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  raw_features[f"{tickers[k]}_cos{k}"] = (
C:\Users\Tienda\AppData\Local\Temp/ipykernel_64644/1060488693.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consid

[[0.0051746557900174 False False ... 0.9668478136052826 0.0 0.0]
 [-0.0110582633067428 True False ... -0.06450844944933187 -0.0 -0.0]
 [-0.0131137799954521 False True ... -0.47495107206703463 0.0 -0.0]
 ...
 [-0.0226914111874445 False True ... 0.8695893893466108 -0.0 0.0]
 [0.0004412738324759 False False ... 0.02151609743623148 0.0 0.0]
 [-0.025935264378727 False True ... 0.357698238833147 0.0 0.0]] 0        0.000895
1        0.004104
2       -0.002486
3        0.002747
4        0.003817
           ...   
30444   -0.061265
30445   -0.002704
30446   -0.028762
30447    0.000027
30448   -0.016565
Name: alpha_expost, Length: 30449, dtype: float64
Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 4.2665 - acc: 0.0000e+00 - val_loss: 1.7971 - val_acc: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 118ms/step - loss: 3.0045 - acc: 0.0000e+00 - val_loss: 1.3899 - val_acc: 0.0000e+00
Epoch 3/200
1/1 [==============================] - 0s 137ms/step - loss: 2